# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

In [ ]:
domain_size=False
forcing_source = False
forcing_sampling_freq = True

In [ ]:
if domain_size:
    date_min = '2010-01-01'
    date_max = '2014-12-31'
elif forcing_source:
    date_min = '2010-01-01'
    date_max = '2022-12-31'
elif forcing_sampling_freq:
    date_min = '2013-01-01'
    date_max = '2013-12-31'

In [ ]:
if domain_size:
    lon_min = -24
    lon_max = 20
    lat_min = 20
    lat_max = 56
if forcing_source:
    lon_min = -13.5
    lon_max = 6.5
    lat_min = 32
    lat_max = 49
if forcing_sampling_freq:
    lon_min = -19
    lon_max = 12
    lat_min = 28
    lat_max = 54

## Format functions

In [ ]:
def format_LMDZ_MO(filename, name, color=None, date_min=None, date_max=None):
    """
    Format the LMDZ-MO output for plotting.
    """
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name
    ds.attrs['plot_color'] = color if color else 'blue'

    rename_dict = {'time_counter':'time'}
    ds = ds.rename(rename_dict)
    #restrict
    if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
        ds = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    else:
        print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')
    if date_min is not None and date_max is not None:
        ds = ds.sel(time=slice(date_min, date_max))
    else:
        print('No date restriction applied, missing date_min, date_max')

    #edit and convert units
    ds['evap'] = ds['evap'] *3600 * 24
    ds['evap'].attrs['units'] = 'mm/d'
    ds['evap'].attrs['name'] = 'ET'

    ds['precip'] = ds['precip'] *3600 * 24
    ds['precip'].attrs['units'] = 'mm/d'
    ds['precip'].attrs['name'] = 'Precipitation'

    ds['t2m'] = ds['t2m'] - 273.15
    ds['t2m'].attrs['units'] = '°C'
    ds['t2m'].attrs['name'] = '2-m temperature'

    ds['fluxsens']= -ds['sens']
    ds['fluxsens'].attrs['units'] = 'W/m²'
    ds['fluxsens'].attrs['name'] = 'Sensible heat flux'

    ds['netrad'] = ds['LWdnSFC'] - ds['LWupSFC'] + ds['SWdnSFC'] - ds['SWupSFC']
    ds['netrad'].attrs['name'] = 'Surface net radiation'
    ds['netrad'].attrs['units'] = 'W/m²'

    ds['SWnetSFC'] = ds['SWdnSFC'] - ds['SWupSFC']
    ds['SWnetSFC'].attrs['name'] = 'Surface net SW'
    ds['SWnetSFC'].attrs['units'] = 'W/m²'

    ds['LWnetSFC'] = ds['LWdnSFC'] - ds['LWupSFC']
    ds['LWnetSFC'].attrs['name'] = 'Surface net LW'
    ds['LWnetSFC'].attrs['units'] = 'W/m²'

    ds['P - E'] = ds['precip'] - ds['evap']
    ds['P - E'].attrs['units'] = 'mm/d'

    #add names and units
    ds['SWdnSFC'].attrs['name'] = 'Surface SW down'
    ds['SWdnSFC'].attrs['units'] = 'W/m²'
    ds['LWdnSFC'].attrs['name'] = 'Surface LW down'
    ds['LWdnSFC'].attrs['units'] = 'W/m²'

    ds['cldt'].attrs['name'] = 'Total cloud cover'
    ds['cldt'].attrs['units'] = '%'

    ds['cldl'] = ds['cldl'] * 100  # Convert to percentage
    ds['cldl'].attrs['name'] = 'Low cloud cover'
    ds['cldl'].attrs['units'] = '%'

    ds['cldm'] = ds['cldm'] * 100  # Convert to percentage
    ds['cldm'].attrs['name'] = 'Medium cloud cover'
    ds['cldm'].attrs['units'] = '%'

    ds['cldh'] = ds['cldh'] * 100  # Convert to percentage
    ds['cldh'].attrs['name'] = 'High cloud cover'
    ds['cldh'].attrs['units'] = '%'

    return ds

## Obs

### ERA5

In [ ]:
#open ERA5 file
filename = '../../../obs/ERA5/ERA5_large_*.nc'
era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
era.attrs['plot_color'] = 'black'

#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'tp':'precip',
        'ro':'totrunoff',
        'u10':'u10m',
        'v10':'v10m',
        'p71.162' : 'uq',
        'p72.162' : 'vq',
        'sshf' : 'fluxsens',
        'slhf' : 'fluxlat',
        'si10' : 'sfcWind',
        'ssrd' : 'SWdnSFC',
        'strd' : 'LWdnSFC',
        'ssr' : 'SWnetSFC',
        'str' : 'LWnetSFC',
        'tcc' : 'cldt',
        'lcc' : 'cldl',
        'mcc' : 'cldm',
        'hcc' : 'cldh'
        }
era = era.rename(dict)

#restrict
if date_min is not None and date_max is not None:
        era=era.sel(time=slice(date_min, date_max))
else:
        print('No date restriction applied, missing date_min, date_max')
if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
        era = era.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max, lat_min))
else:
        print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'

era['cldt'] = era['cldt']*100
era['cldt'].attrs['units'] = '%'
era['cldl'] = era['cldl']*100
era['cldl'].attrs['units'] = '%'
era['cldm'] = era['cldm']*100
era['cldm'].attrs['units'] = '%'
era['cldh'] = era['cldh']*100
era['cldh'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era['calcWind'] = np.sqrt(era['u10m']**2 + era['v10m']**2)
era['calcWind'].attrs['units'] = 'm/s'

era['t2m'] = era['t2m'] - 273.15
era['t2m'].attrs['units'] = '°C'
era['t2m'].attrs['name'] = '2-m temperature'
   
#convert 4 radiation variables from J/m2 to W/m²
era['SWdnSFC'] = era['SWdnSFC'] / (3600 *24)
era['SWdnSFC'].attrs['units'] = 'W/m²'
era['LWdnSFC'] = era['LWdnSFC'] / (3600 *24)
era['LWdnSFC'].attrs['units'] = 'W/m²'
era['SWnetSFC'] = era['SWnetSFC'] / (3600 *24)
era['SWnetSFC'].attrs['units'] = 'W/m²'
era['LWnetSFC'] = era['LWnetSFC'] / (3600 *24)
era['LWnetSFC'].attrs['units'] = 'W/m²'
#add up vars and netrad
era['LWupSFC'] = - era['LWnetSFC'] + era['LWdnSFC']
era['LWupSFC'].attrs['units'] = 'W/m²'
era['SWupSFC'] = - era['SWnetSFC'] + era['SWdnSFC']
era['SWupSFC'].attrs['units'] = 'W/m²'
era['netrad'] = era['LWnetSFC'] + era['SWnetSFC']
era['netrad'].attrs['units'] = 'W/m²'

#convert fluxsens and fluxlat to W/m²
era['fluxsens'] = -era['fluxsens'] / (3600 *24)
era['fluxsens'].attrs['units'] = 'W/m²'
era['fluxlat'] = -era['fluxlat'] / (3600 *24)
era['fluxlat'].attrs['units'] = 'W/m²'

era


In [ ]:
#open tqruv file
filename='../../../obs/ERA5/TQRUV_850_2010_2022_monthly.nc'
tqruv = xr.open_dataset(filename)
tqruv.attrs['name'] = 'ERA5'
#rename longitude and latitude to lon and lat
tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

#restrict
if date_min is not None and date_max is not None:
    tqruv = tqruv.sel(time=slice(date_min, date_max))
else:
    print('No date restriction applied, missing date_min, date_max')
if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
    tqruv = tqruv.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
else:
    print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')


#rename all variables to add 850 in name
tqruv['u850'] = tqruv['u']
tqruv['v850'] = tqruv['v']
tqruv['q850'] = tqruv['q']
tqruv['t850'] = tqruv['t']
tqruv['r850'] = tqruv['r']
tqruv

### GLEAM

In [ ]:
#gleam
# filename='../../../obs/GLEAMv4.1a/E_1980-2022_GLEAM_v3.8a_MO.nc'
filename='../../../obs/GLEAMv4.1a/E/*_MO.nc'
gleam=xr.open_mfdataset(filename)
gleam.attrs['name']='GLEAM'
gleam.attrs['plot_color'] = 'black'

#restrict
if date_min is not None and date_max is not None:
    gleam = gleam.sel(time=slice(date_min, date_max))
else:
    print('No date restriction applied, missing date_min, date_max')
if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
    gleam = gleam.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
else:
    print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')

#name gleam vars like in LMDZOR outputs
dict =  {
        'E':'evap'
        }
gleam = gleam.rename(dict)
#convert mm/month to mm/day
gleam['evap']=gleam['evap']/30
gleam['evap'].attrs['units']='mm/d'
gleam

### FluxCom

In [ ]:
#FluxCom
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom=xr.open_mfdataset(filename)
fluxcom.attrs['name']='FluxCom'
fluxcom.attrs['plot_color'] = 'black'

#restrict
if date_min is not None and date_max is not None:
    fluxcom = fluxcom.sel(time=slice(date_min, date_max))
else:
    print('No date restriction applied, missing date_min, date_max')
if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
    fluxcom = fluxcom.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))
else:
    print('No domain restriction applied, missing lon_min, lon_max, lat_min, lat_max')

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m²
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m²
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m²
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

### GPCC

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc=xr.open_mfdataset(filename)
gpcc.attrs['name'] = 'GPCC'
gpcc.attrs['plot_color'] = 'black'
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc = gpcc.sortby('lon')

#restrict
if date_min is not None and date_max is not None:
    gpcc = gpcc.sel(time=slice(date_min, date_max))
else:
    print('No date restriction applied, missing date_min, date_max')        
if lon_min is not None and lon_max is not None and lat_min is not None and lat_max is not None:
    gpcc = gpcc.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max,lat_min))
# gpcc2=gpcc.sel(lon=slice(0,lon_max+0.25),lat=slice(lat_max,lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < lon_max+0.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25))

# gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
# gpcc['precip'].attrs['units'] = 'mm/d'
gpcc['precip'] = convert_mm_per_month_to_mm_per_day(gpcc['precip'])
gpcc

## Sims 3 domain sizes 

2010-2014

In [ ]:
LAM_1000_40_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# LAM_1500_40_dir='../../../JZ_simu_outputs/LAM/LAM_1500_40_sim224'
LAM_1500_60_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/LAM_1500_60_sim206' #not the exact version used afterwards but appropriate for sensitivity comparison on domain size
LAM_2000_80_dir='../../../JZ_simu_outputs/LAM/LAM_2000_80/LAM_2000_80_sim199'

In [ ]:
#smaller LAM (original)
if domain_size:
    filename = '{}/ATM/TS*.nc'.format(LAM_1000_40_dir)
    # filename = '{}/ATM/MO/*.nc'.format(LAM_1000_40_dir)
    color = 'blue'
    lmdz_1000_40 = format_LMDZ_MO(filename, 'LAM_1000km_NBP40', color=color, date_min=date_min, date_max=date_max)

# lmdz_1000_40


In [ ]:
if domain_size:
    filename = '{}/ATM/MO/*.nc'.format(LAM_1500_60_dir)
    lmdz_1500_60 = format_LMDZ_MO(filename, 'LAM_1500km_NBP60', color='green')

# lmdz_1500_60

In [ ]:
if domain_size:
    filename = '{}/ATM/MO/*.nc'.format(LAM_2000_80_dir)
    lmdz_2000_80 = format_LMDZ_MO(filename, 'LAM_2000km_NBP80', color='red')
# lmdz_2000_80

### Masks and interp

In [ ]:
if domain_size:
    #define masks
    con_mask=lmdz_2000_80['contfracATM']>0.95
    ip_mask=polygon_to_mask(lmdz_2000_80, iberian_peninsula)
    
    # cont_lmdz_1000_40=lmdz_1000_40_i80.where(con_mask)
    # cont_lmdz_1500_60=lmdz_1500_60_i80.where(con_mask)
    cont_lmdz_2000_80=lmdz_2000_80.where(con_mask)

    # ip_lmdz_1000_40 = lmdz_1000_40_i80.where(ip_mask, drop=False).where(con_mask)
    # ip_lmdz_1500_60 = lmdz_1500_60_i80.where(ip_mask, drop=False).where(con_mask)
    ip_lmdz_2000_80 = lmdz_2000_80.where(ip_mask, drop=False).where(con_mask)


In [ ]:
ip_interp=False
if domain_size:
    lmdz_1000_40_era = lmdz_1000_40.interp_like(era)
    # lmdz_1000_40_gpcc = lmdz_1000_40.interp_like(gpcc)
    # lmdz_1000_40_gleam = lmdz_1000_40.interp_like(gleam)
    # lmdz_1000_40_fluxcom = lmdz_1000_40.interp_like(fluxcom)
    # lmdz_1000_40_tqruv = lmdz_1000_40.interp_like(tqruv)

    lmdz_1500_60_era = lmdz_1500_60.interp_like(era)
    # lmdz_1500_60_gpcc = lmdz_1500_60.interp_like(gpcc)
    # lmdz_1500_60_gleam = lmdz_1500_60.interp_like(gleam)
    # lmdz_1500_60_fluxcom = lmdz_1500_60.interp_like(fluxcom)
    # lmdz_1500_60_tqruv = lmdz_1500_60.interp_like(tqruv)

    lmdz_2000_80_era = lmdz_2000_80.interp_like(era)
    # lmdz_2000_80_gpcc = lmdz_2000_80.interp_like(gpcc)
    # lmdz_2000_80_gleam = lmdz_2000_80.interp_like(gleam)
    # lmdz_2000_80_fluxcom = lmdz_2000_80.interp_like(fluxcom)
    # lmdz_2000_80_tqruv = lmdz_2000_80.interp_like(tqruv)

    if ip_interp:
        ip_lmdz_1000_40_era = ip_lmdz_1000_40.interp_like(era)
        # ip_lmdz_1000_40_gpcc = ip_lmdz_1000_40.interp_like(gpcc)
        # ip_lmdz_1000_40_gleam = ip_lmdz_1000_40.interp_like(gleam)
        # ip_lmdz_1000_40_fluxcom = ip_lmdz_1000_40.interp_like(fluxcom)
        # ip_lmdz_1000_40_tqruv = ip_lmdz_1000_40.interp_like(tqruv)

        ip_lmdz_1500_60_era = ip_lmdz_1500_60.interp_like(era)
        # ip_lmdz_1500_60_gpcc = ip_lmdz_1500_60.interp_like(gpcc)
        # ip_lmdz_1500_60_gleam = ip_lmdz_1500_60.interp_like(gleam)
        # ip_lmdz_1500_60_fluxcom = ip_lmdz_1500_60.interp_like(fluxcom)
        # ip_lmdz_1500_60_tqruv = ip_lmdz_1500_60.interp_like(tqruv)

        ip_lmdz_2000_80_era = ip_lmdz_2000_80.interp_like(era)
        # ip_lmdz_2000_80_gpcc = ip_lmdz_2000_80.interp_like(gpcc)
        # ip_lmdz_2000_80_gleam = ip_lmdz_2000_80.interp_like(gleam)
        # ip_lmdz_2000_80_fluxcom = ip_lmdz_2000_80.interp_like(fluxcom)
        # ip_lmdz_2000_80_tqruv = ip_lmdz_2000_80.interp_like(tqruv)

## Forcing with LMDZ vs forcing with ERA5

LAM 1000km NBP40


2010-2022 (?)

In [ ]:
ICOLMDZ_dir ='../../../JZ_simu_outputs/LAM/LAM_1000_40/CM6'

In [ ]:
#LMDZ
if forcing_source:
    filename = '{}/ATM/*.nc'.format(LAM_1000_40_dir)
    lmdz_era = format_LMDZ_MO(filename, 'forced_by_ERA', color='red', date_min=date_min, date_max=date_max)

    filename = '{}/ATM/*.nc'.format(ICOLMDZ_dir)
    lmdz_ico = format_LMDZ_MO(filename, 'forced_by_ICOLMDZ', color='blue', date_min=date_min, date_max=date_max)
    #convert cloud cover (not same unit as in initial sim...?)
    lmdz_ico['cldt'] = lmdz_ico['cldt'] * 100  # Convert to percentage
    lmdz_ico['cldt'].attrs['name'] = 'Total cloud cover'
    lmdz_ico['cldt'].attrs['units'] = '%'

### Masks and interp

In [ ]:
if forcing_source:
    lmdz_ico_iera = lmdz_ico.interp_like(era)
    lmdz_era_iera = lmdz_era.interp_like(era)

In [ ]:
ip_masking=False
if forcing_source and ip_masking:
    con_mask=lmdz_era_iera['contfracATM']>0.95
    ip_mask=polygon_to_mask(lmdz_era_iera, iberian_peninsula)
    ip_lmdz_ico_iera = lmdz_ico_iera.where(ip_mask, drop=False).where(con_mask)
    ip_lmdz_era_iera = lmdz_era_iera.where(ip_mask, drop=False).where(con_mask)
    ip_era = era.where(ip_mask, drop=False).where(con_mask)

## Forcing 1h or 6h

LAM 1500km NBP60

2013 played 10 times

In [ ]:
onehour_dir ='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr_1h'
sixhour_dir ='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr_6h'

In [ ]:
if forcing_sampling_freq:
    simlist=['312', '3122', '3123', '3124', '3125', '3126', '3127', '3128', '3129', '247']
    sims_1h=[]
    for simnb in simlist:
        filename = '{}/sim{}_2013*.nc'.format(onehour_dir, simnb)
        # print(filename)
        lmdz_1h_tmp = format_LMDZ_MO(filename, 'forcing_1h', color='blue')
        #add dimension simnb 
        lmdz_1h_tmp['simnb'] = simnb
        # make it a coordinate
        lmdz_1h_tmp = lmdz_1h_tmp.assign_coords(simnb=('simnb', [simnb]))
        #add to list
        sims_1h.append(lmdz_1h_tmp)
    lmdz_1h = xr.concat(sims_1h, dim='simnb')
    #mean over simnb
    lmdz_1h_mean = mean_dataset(lmdz_1h,mean_dim='simnb')
    lmdz_1h_mean = transfer_vars(lmdz_1h, lmdz_1h_mean)
    

In [ ]:
if forcing_sampling_freq:
    simlist = ['310', '3101', '3102', '3103', '3104', '311', '3111','3112', '3113', '306']
    sims_6h = []
    for simnb in simlist:
        filename = '{}/sim{}_2013*.nc'.format(sixhour_dir, simnb)
        # print(filename)
        lmdz_6h_tmp = format_LMDZ_MO(filename, 'forcing_6h', color='red')
        #add dimension simnb
        lmdz_6h_tmp['simnb'] = simnb
        #make it a coordinate
        lmdz_6h_tmp = lmdz_6h_tmp.assign_coords(simnb=('simnb', [simnb]))
        #append to list
        sims_6h.append(lmdz_6h_tmp)
    lmdz_6h = xr.concat(sims_6h, dim='simnb')
    #mean over simnb
    lmdz_6h_mean = mean_dataset(lmdz_6h,mean_dim='simnb')
    lmdz_6h_mean=transfer_vars(lmdz_6h, lmdz_6h_mean)

In [ ]:
if forcing_sampling_freq:
    lmdz_1h_mean_iera = lmdz_1h_mean.interp_like(era)
    lmdz_6h_mean_iera = lmdz_6h_mean.interp_like(era)

In [ ]:
if forcing_sampling_freq:
    lon_min = -19
    lon_max = 12
    lat_min = 28
    lat_max = 54
    era = era.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))
    lmdz_1h_mean_iera = lmdz_1h_mean_iera.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))
    lmdz_6h_mean_iera = lmdz_6h_mean_iera.sel(lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min))

# Tests

## Maps

In [ ]:
#Display map of var for ds
var=''
ds=lmdz_ico_iera
# ds=era
color_map=reds
min_value=0
max_value=100
min_value, max_value = None, None

map_ave(ds, var, cmap=color_map, vmin=min_value, vmax=max_value, hex=False)

In [ ]:
#Diff map
var='precip'
ds1=lmdz_ico_iera
ds2=lmdz_era_iera
max_value= 2
min_value=-2
# max_value, min_value = None, None

color_map=emb

# title='Evapotranspiration bias (mm/d, {} - {})'.format( ds1.attrs['name'], ds2.attrs['name'])
title=None

map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, cmap=color_map, hex=False, sig=False)

# diff=ds1[var]-ds2[var]
# title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
# map_seasons(diff, cmap=color_map,  vmin=min_value, vmax=max_value, title=title, hex=False)

In [ ]:
#map for 4 seasons
var='precip'
ds1=cont_sim
ds2=cont_era
max_value=2
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'precip'
min_value=None
max_value=6
color_map=wet
ds1=era
ds2=lmdz_1500_60
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map, hex=True)

In [ ]:
#Display map of var for ds on restricted area
var='precip'
ds = lmdz_2000_80_era
latmin=42.2
latmax=43.5
lonmin=-2
lonmax=3
vmax= 3
vmin=-3

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time') -era[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb, vmin=vmin, vmax=vmax, title=ds.attrs['name'])

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

In [ ]:
ds=sim
map_wind(ds, height='10m', scale=50)

In [ ]:
ds=sim_era
map_moisture_transport(ds, scale=1500)

In [ ]:
ds=geopt
plotvar = ds['z']
map_plotvar(plotvar, cmap='terrain', vmin=0.0, vmax=2400, title='Geopotential height (m)')

## Time series

In [ ]:
var='precip'

ds1=lmdz_1h_mean
ds2=lmdz_6h_mean
ds3=era

ds_list=[ds1, ds2, ds3]

year_max=2022
time_series_ave(ds_list, var, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']),
                ds_colors=True)
seasonal_cycle_ave(ds_list, var, year_max=year_max, 
                   title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']),
                   ds_colors=True)

# Figures

## Domain size

### Var maps in ERA

In [ ]:
savefig=True
#Var map
vars=['precip', 'evap', 'LWdnSFC','SWdnSFC', 'cldt', 'cldl']
vmins=[0, 0, 250, 130, 0, 0]
vmaxs=[4, 4, 370, 250, 100, 100]
cmaps=[bluesW, bluesW, reds, reds, greys, greys]
ds_list=[era]#, lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]

title='off'

i=0
for var in vars:
    label='{} ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units'])
    for ds1 in ds_list:
        map_ave(ds1, var, vmin=vmins[i], vmax=vmaxs[i], title=title, clabel=label, cmap=cmaps[i])
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/var_map_{}_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)
    i+=1

### 3 maps for 3 sizes, whole domain, diff with ERA5 : precip, evap

In [ ]:
savefig=True
#Diff map
vars=['precip', 'evap']
max_value= 1.5
min_value=-1.5
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb_neutral

title='off'
# title=None

for var in vars:
    label='{} difference ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units'])
    for ds1 in ds_list:
        map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map, sig=False)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
#relative diff map
vars=['precip', 'evap']
max_value= 100
min_value=-100
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb_neutral

title='off'

for var in vars:
    label='{} relative difference (%)'.format(lmdz_1000_40_era[var].attrs['name'])
    for ds1 in ds_list:
        map_rel_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/rel_diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

### Diffs for radiative components

In [ ]:
savefig=True
#Diff map
vars=['SWdnSFC', 'LWdnSFC']
min_value=-30
max_value= 30
# min_value, max_value = None, None

ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb

title='off'

for var in vars:
    label='{} difference ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units'])
    for ds1 in ds_list:
        map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map, sig=False)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
#Relative diff map
vars=['SWdnSFC', 'LWdnSFC']
max_value= 20
min_value=-20
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb

title='off'

for var in vars:
    label='{} relative diff. (%)'.format(lmdz_1000_40_era[var].attrs['name'])
    for ds1 in ds_list:
        map_rel_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/rel_diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

### Diffs for other vars

In [ ]:
savefig=True

varlist=['t2m', 'fluxsens', 'netrad', 'SWnetSFC', 'LWnetSFC']
vmaxs=[2, 50, 25, 25, 25]
cmaps=[emb, emb, emb, emb, emb]

ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era

i=0
for var in varlist:
    for ds1 in ds_list:
        clabel='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])

        if savefig:
            title='off'
        else:
            title='{} bias to ERA5'.format(var)
        map_diff_ave(ds1, ds2, var, vmin=-vmaxs[i], vmax=vmaxs[i],
                    title=title, cmap=cmaps[i], clabel=clabel,
                        sig=False)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)
    i+=1

In [ ]:
savefig=True

varlist=['cldt', 'cldl', 'cldm', 'cldh']
#add names to lmdz_1000_40_era
lmdz_1000_40_era['cldl'].attrs['name'] = 'Low cloud cover'
lmdz_1000_40_era['cldm'].attrs['name'] = 'Medium cloud cover'
lmdz_1000_40_era['cldh'].attrs['name'] = 'High cloud cover'

vmax= 25
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era

i=0
for var in varlist:
    clabel='{} difference ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units'])
    for ds1 in ds_list:

        if savefig:
            title='off'
        else:
            title='{} bias to ERA5'.format(var)
        map_diff_ave(ds1, ds2, var, vmin=-vmax, vmax=vmax,
                    title=title, cmap=emb, clabel=clabel)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)
    i+=1

In [ ]:
savefig=True
#rel diff maps
varlist=['cldt', 'cldl', 'cldm', 'cldh']

vmax= 50
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era

i=0
for var in varlist:
    if savefig:
        title='off'
    else:
        title='{} bias to ERA5'.format(var)

    clabel='{} relative diff. ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units']) 

    for ds1 in ds_list:
        map_rel_diff_ave(ds1, ds2, var, vmin=-vmax, vmax=vmax,
                    title=title, cmap=emb, clabel=clabel)
        if savefig and domain_size:
            plt.savefig('figures/chap4/domain_size/rel_diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)
    i+=1

## Forcing source

### Map of diff in multiple vars for LMDZ_ico and LMDZ_era

In [ ]:
savefig=True
varlist=['precip', 'evap', 'LWdnSFC', 'SWdnSFC']#, 't2m', 'fluxsens', 'netrad', 'SWnetSFC', 'LWnetSFC']
vmaxs=[2, 2, 25, 25, 2, 50, 25, 25, 25]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb, emb, emb, emb, emb, emb]
ds1=lmdz_era_iera
ds2=lmdz_ico_iera
ds3=era

i=0
for var in varlist:
    clabel='{} difference ({})'.format(ds2[var].attrs['name'], ds1[var].attrs['units'])

    #LMDZ_era
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_era - ERA5)'.format(var)
    map_diff_ave(ds1, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i],
                 title=title, cmap=cmaps[i], clabel=clabel,
                    sig=False)
    if savefig and forcing_source:
        plt.savefig('figures/chap4/forcing_source/diff_map_{}_era_era.png'.format(var), bbox_inches='tight', dpi=300)

    #LMDZ_ico
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_ico - ERA5)'.format(var)
    map_diff_ave(ds2, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i], 
                 title=title, cmap=cmaps[i], clabel=clabel,
                  sig=False)
    if savefig and forcing_source:
        plt.savefig('figures/chap4/forcing_source/diff_map_{}_ico_era.png'.format(var), bbox_inches='tight', dpi=300)
    
    i+=1

### Cloud variables

In [ ]:
savefig=True
varlist=['cldt', 'cldl', 'cldm', 'cldh']
vmaxs=[25, 25, 25, 25]
cmaps=[emb, emb, emb, emb]
ds1=lmdz_era_iera
ds2=lmdz_ico_iera
ds3=era

#add var names to lmdz_era_iera
# lmdz_era_iera['cldl'].attrs['name'] = 'Low cloud cover'
# lmdz_era_iera['cldm'].attrs['name'] = 'Medium cloud cover'
# lmdz_era_iera['cldh'].attrs['name'] = 'High cloud cover'

i=0
for var in varlist:
    clabel='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])

    #LMDZ_era
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_era - ERA5)'.format(var)
    map_diff_ave(ds1, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i],
                 title=title, cmap=cmaps[i], clabel=clabel,
                    sig=False)
    if savefig and forcing_source:
        plt.savefig('figures/chap4/forcing_source/diff_map_{}_era_era.png'.format(var), bbox_inches='tight', dpi=300)

    #LMDZ_ico
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_ico - ERA5)'.format(var)
    map_diff_ave(ds2, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i], 
                 title=title, cmap=cmaps[i], clabel=clabel,
                  sig=False)
    if savefig and forcing_source:
        plt.savefig('figures/chap4/forcing_source/diff_map_{}_ico_era.png'.format(var), bbox_inches='tight', dpi=300)
    
    i+=1

### Times series over IP

t2m (ERA5 as ref), precip (GPCC as ref), evap (GLEAM as ref)

## Forcing sampling frequency

### Maps of diff in multiple vars for 1h and 6h

In [ ]:
savefig=True
varlist=['precip', 'evap', 'LWdnSFC', 'SWdnSFC', 't2m', 'fluxsens', 'netrad', 'SWnetSFC', 'LWnetSFC']
vmaxs=[2, 2, 25, 25, 2, 50, 25, 25, 25]
cmaps=[emb_neutral, emb_neutral, emb, emb, emb, emb, emb, emb, emb, emb]
ds1=lmdz_1h_mean_iera
ds2=lmdz_6h_mean_iera
ds3=era

i=0
for var in varlist:
    clabel='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])

    #LMDZ_1h
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_1h - ERA5)'.format(var)
    map_diff_ave(ds1, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i],
                 title=title, cmap=cmaps[i], clabel=clabel,
                    sig=False)
    if savefig and forcing_sampling_freq:
        plt.savefig('figures/chap4/forcing_sampling_freq/diff_map_{}_lmdz1h_era.png'.format(var), bbox_inches='tight', dpi=300)

    #LMDZ_ico
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_6h - ERA5)'.format(var)
    map_diff_ave(ds2, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i], 
                 title=title, cmap=cmaps[i], clabel=clabel,
                  sig=False)
    if savefig and forcing_sampling_freq:
        plt.savefig('figures/chap4/forcing_sampling_freq/diff_map_{}_lmdz6h_era.png'.format(var), bbox_inches='tight', dpi=300)
    
    i+=1

### Cloud variables

In [ ]:
savefig=True
varlist=['cldt', 'cldl', 'cldm', 'cldh']
vmaxs=[25, 25, 25, 25]
cmaps=[emb, emb, emb, emb]
ds1=lmdz_1h_mean_iera
ds2=lmdz_6h_mean_iera
ds3=era

i=0
for var in varlist:
    clabel='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])

    #LMDZ_1h
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_1h - ERA5)'.format(var)
    map_diff_ave(ds1, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i],
                 title=title, cmap=cmaps[i], clabel=clabel,
                    sig=False)
    if savefig and forcing_sampling_freq:
        plt.savefig('figures/chap4/forcing_sampling_freq/diff_map_{}_lmdz1h_era.png'.format(var), bbox_inches='tight', dpi=300)

    #LMDZ_ico
    if savefig:
        title='off'
    else:
        title='{} bias (lmdz_6h - ERA5)'.format(var)
    map_diff_ave(ds2, ds3, var, vmin=-vmaxs[i], vmax=vmaxs[i], 
                 title=title, cmap=cmaps[i], clabel=clabel,
                  sig=False)
    if savefig and forcing_sampling_freq:
        plt.savefig('figures/chap4/forcing_sampling_freq/diff_map_{}_lmdz6h_era.png'.format(var), bbox_inches='tight', dpi=300)
    
    i+=1

### Times series over IP

t2m (ERA5 as ref), precip (GPCC as ref), evap (GLEAM as ref)